In [0]:
%sql
USE CATALOG nyc_taxi;
USE SCHEMA nyc_taxi_schema;

In [0]:
df = spark.table("nyc_taxi.nyc_taxi_schema.yellow_trips_csv_v")
df.createOrReplaceTempView("table")

In [0]:
%sql
-- Q1.1
SELECT * FROM table
WHERE fare_amount > 1000;

In [0]:
%sql
-- Q1.2
SELECT * FROM table
WHERE fare_amount <= 0

In [0]:
%sql
-- Q2.1
SELECT corr(fare_amount, trip_distance) AS corr_fare_trip, corr(total_amount, trip_distance) AS corr_total_trip 
FROM table
WHERE trip_distance BETWEEN 0.1 AND 75
AND fare_amount > 0
AND fare_amount <1000
AND total_amount BETWEEN 0.1 AND 1100;

Q2.2 explanation


In [0]:
%sql
-- Q3
SELECT AVG(datediff(minute,cast(tpep_pickup_datetime as timestamp), cast(tpep_dropoff_datetime as timestamp))) AS trip_duration, passenger_count
FROM table 
GROUP BY passenger_count;


In [0]:
%sql
-- Q4, Q5
SELECT
CASE 
WHEN trip_distance <1 THEN '0-1 Miles'
WHEN trip_distance < 2 THEN '1-2 Miles'
WHEN trip_distance < 5 THEN '2-5 Miles'
ELSE '5+ Miles'
END AS distance_bins,
AVG(fare_amount) AS avg_fare_by_bin
FROM table
GROUP BY distance_bins
ORDER BY MIN(trip_distance);

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Q6
SELECT passenger_count, COUNT(*) AS Number_of_Trips, AVG(fare_amount) AS avg_fare_by_passengers
FROM table 
GROUP BY passenger_count
ORDER BY passenger_count;

In [0]:
%sql
-- Q7
SELECT ROUND(pickup_longitude, 3), ROUND(pickup_latitude, 3), COUNT(*) AS pickups
FROM table 
WHERE pickup_longitude != 0
AND pickup_latitude != 0
GROUP BY ROUND(pickup_longitude, 3), ROUND(pickup_latitude, 3)
ORDER BY pickups DESC
LIMIT 10;

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- initialize weather data
USE CATALOG weather_data;
USE SCHEMA weather_schema;

In [0]:
weather_df = spark.table("weather_data.weather_schema.weather_csv_v")
weather_df.createOrReplaceTempView("weather")

In [0]:
%sql
-- Q16.1
CREATE OR REPLACE TEMP VIEW taxi_with_date AS
SELECT *,
       DATE(tpep_pickup_datetime) AS trip_date
FROM table
WHERE fare_amount > 0 AND trip_distance > 0;


In [0]:
%sql
-- Q16.1
CREATE OR REPLACE TEMP VIEW joined_weather AS
SELECT 
    t.trip_date,
    AVG(t.fare_amount) AS avg_fare,
    AVG(w.TMIN) AS avg_min_temp,
    AVG(w.TMAX) AS avg_max_temp,
    AVG(w.PRCP) AS avg_precip
FROM taxi_with_date t
LEFT JOIN weather w
ON t.trip_date = w.DATE
GROUP BY t.trip_date
ORDER BY t.trip_date;


In [0]:
%sql
-- Q16.1
CREATE OR REPLACE TEMP VIEW weather_summary AS
SELECT
  CASE WHEN avg_precip > 0 THEN 'Rainy' ELSE 'Clear' END AS weather_condition,
  AVG(avg_fare) AS avg_fare
FROM joined_weather
GROUP BY weather_condition;


In [0]:
%sql
-- Q16.2
SELECT * FROM weather_summary;

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Q17.1
CREATE OR REPLACE TEMP VIEW passenger_by_hour AS
SELECT
  HOUR(tpep_pickup_datetime) AS hour,
  AVG(passenger_count) AS avg_passengers
FROM table
WHERE passenger_count > 0
GROUP BY hour
ORDER BY hour;

In [0]:
%sql
-- Q17.2
SELECT * FROM passenger_by_hour

Databricks visualization. Run in Databricks to view.

In [0]:
%sql
-- Q18.1
CREATE OR REPLACE TEMP VIEW revenue_by_day AS
SELECT
  DATE_FORMAT(tpep_pickup_datetime, 'E') AS day_of_week,
  SUM(total_amount) AS total_revenue
FROM table
GROUP BY day_of_week
ORDER BY total_revenue DESC;


In [0]:
%sql
-- Q18.2
SELECT * FROM revenue_by_day

Databricks visualization. Run in Databricks to view.